<a href="https://colab.research.google.com/github/NitCoh/NLPAss2/blob/master/NLP_Ass2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB


In [2]:
!wget 'https://www.cs.bgu.ac.il/~elhadad/nlp20/hw2_1_data.tar.gz'
!tar -xvzf hw2_1_data.tar.gz
!mv data ../
!wget https://www.cs.bgu.ac.il/~elhadad/nlp20/ngram_counts.txt.gz
!mv ngram_counts.txt.gz ../data/

--2020-01-16 13:04:31--  https://www.cs.bgu.ac.il/~elhadad/nlp20/hw2_1_data.tar.gz
Resolving www.cs.bgu.ac.il (www.cs.bgu.ac.il)... 132.72.41.239
Connecting to www.cs.bgu.ac.il (www.cs.bgu.ac.il)|132.72.41.239|:443... connected.
ERROR: cannot verify www.cs.bgu.ac.il's certificate, issued by ‘CN=Let's Encrypt Authority X3,O=Let's Encrypt,C=US’:
  Unable to locally verify the issuer's authority.
To connect to www.cs.bgu.ac.il insecurely, use `--no-check-certificate'.
tar (child): hw2_1_data.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
mv: cannot stat 'data': No such file or directory
--2020-01-16 13:04:42--  https://www.cs.bgu.ac.il/~elhadad/nlp20/ngram_counts.txt.gz
Resolving www.cs.bgu.ac.il (www.cs.bgu.ac.il)... 132.72.41.239
Connecting to www.cs.bgu.ac.il (www.cs.bgu.ac.il)|132.72.41.239|:443... connected.
ERROR: cannot verify www.cs.bgu.ac.il's certificate, is

In [3]:
#############################################################
## ASSIGNMENT 2_1 CODE SKELETON
#############################################################

from collections import defaultdict
import gzip

#### Q1.1 Evaluation Metrics ####

## Input: y_pred, a list of length n with the predicted labels,
## y_true, a list of length n with the true labels

data_path = '../data'

## Calculates the precision of the predicted labels
def get_precision(y_pred, y_true):
  # For this assignment, complex words 1 are considered positive examples, and simple words 0 are considered negative examples.
    y_pred = [round(y) for y in y_pred]
    tp = len([x for x,y in zip(y_pred,y_true) if x==y==1])
    fp = len([x for x,y in zip(y_pred,y_true) if y==0 and x==1])
    precision = tp/(tp+fp)
    return precision
    
## Calculates the recall of the predicted labels
def get_recall(y_pred, y_true):
  
    ## YOUR CODE HERE...
    y_pred = [round(y) for y in y_pred]
    tp = len([x for x,y in zip(y_pred,y_true) if x==y==1])
    fn = len([x for x,y in zip(y_pred,y_true) if y==1 and x==0])
    recall = tp/(tp+fn)
    return recall

## Calculates the f-score of the predicted labels
def get_fscore(y_pred, y_true):
    ## YOUR CODE HERE...
    p = get_precision(y_pred,y_true)
    r = get_recall(y_pred,y_true)
    fscore = 2*p*r / (p +r)
    return fscore

def test_predictions(y_pred,y_true):
    p = get_precision(y_pred,y_true)
    r = get_recall(y_pred,y_true)
    f1 = get_fscore(y_pred,y_true)
    print("Precision:",p)
    print("Recall:",r)
    print("F-score:",f1)
    return p, r, f1

#### 2. Complex Word Identification ####

## Loads in the words and labels of one of the datasets
def load_file(data_file):
    words = []
    labels = []   
    with open(data_file, 'rt', encoding="utf8") as f:
        i = 0
        for line in f:
            if i > 0:
                line_split = line[:-1].split("\t")
                words.append(line_split[0].lower())
                labels.append(int(line_split[1]))
            i += 1
    return words, labels

### 1.2.1: A very simple baseline

## Labels every word complex
def all_complex(data_file):
    ## YOUR CODE HERE...
    _, y_true = load_file(data_file)
    y_pred = [1]*len(y_true)
    p, r, f1 = test_predictions(y_pred,y_true)

    performance = [p, r, f1]
    return performance



def threshold_plot_helper(training_file,development_file,pred,ts_range):
    p = []
    r = []
    f1= []
    train_words,train_labels = load_file(training_file)
    for threshold in ts_range:
      y_pred_train = [int(pred(x,threshold)) for x in train_words]
      p_t, r_t, f1_t = test_predictions(y_pred_train,train_labels)
      p.append(p_t)
      r.append(r_t)
      f1.append(f1_t)
    plt.ylabel('Precision')
    plt.xlabel('Recall')
    plt.plot(r,p)

    best_index =  f1.index(max(f1))
    best_threshold = ts_range[best_index]
    print("Best threshold:",best_threshold)


    dev_words,dev_labels = load_file(development_file)
    y_pred_dev = [int(pred(x,best_threshold)) for x in dev_words]
    dprecision, drecall, dfscore = test_predictions(y_pred_dev,dev_labels)


    training_performance = [p[best_index], r[best_index], f1[best_index]]
    development_performance = [dprecision, drecall, dfscore]
    return training_performance, development_performance

### 1.2.2: Word length thresholding

## Finds the best length threshold by f-score, and uses this threshold to
## classify the training and development set
def word_length_threshold(training_file, development_file):

    func = lambda x,threshold: len(x) >= threshold
    return threshold_plot_helper(training_file,development_file,func,range(5,11))

### 1.2.3: Word frequency thresholding

## Loads Google NGram counts
def load_ngram_counts(ngram_counts_file): 
   counts = defaultdict(int) 
   with gzip.open(ngram_counts_file, 'rt', errors='ignore') as f: 
       for line in f:
           token, count = line.strip().split('\t') 
           if token[0].islower(): 
               counts[token] = int(count) 
   return counts

# Finds the best frequency threshold by f-score, and uses this threshold to
## classify the training and development set
def word_frequency_threshold(training_file, development_file, counts):
    func = lambda word,threshold : counts[word] >= threshold

    return threshold_plot_helper(training_file,development_file,func,range(100,8000,100))

### 1.3.1: Naive Bayes
        
def dataset_params(words,counts):
  mean_length = sum([len(word) for word in words])/len(words)
  std_length = sum([len(word) - mean_length for word in words])/len(words)

  mean_counts = sum([counts[word] for word in words])/len(words)
  std_counts = sum([counts[word] - mean_counts for word in words])/len(words)

  return mean_length,std_length,mean_counts,std_counts

## Trains a Naive Bayes classifier using length and frequency features
def naive_bayes(training_file, development_file, counts):
    

    def get_features(word,params):
      mean_length,std_length,mean_counts,std_counts = params
      l = (len(word) - mean_length) / std_length
      c = (counts[word] - mean_counts) / std_counts
      return [l,c]

    def get_features2(word,params):
      l = len(word)
      c = counts[word]
      return [l,c]


    train_words,train_labels = load_file(training_file)
    params = dataset_params(train_words,counts)
    print("PARAMS:",params)
    X_train = np.array([np.array(get_features(word,params)) for word in train_words])
    y_true_train = np.array(train_labels)
    print("-----------------------------------")
    print(X_train)
    print("-----------------------------------")

    clf = GaussianNB()
    clf.fit(X_train,y_true_train)
    y_pred_train = clf.predict(X_train)
    tprecision, trecall, tfscore = test_predictions(y_pred_train,y_true_train)
    training_performance = [tprecision, trecall, tfscore]

    #dev
    dev_words,dev_labels = load_file(development_file)

    X_test = np.array([np.array(get_features(word,params)) for word in dev_words])
    y_true_test = np.array(dev_labels)

    y_pred_test = clf.predict(X_test)
  
    dprecision, drecall, dfscore = test_predictions(y_pred_test,y_true_test)
    development_performance = [dprecision, drecall, dfscore]
    return training_performance, development_performance

if __name__ == "__main__":
    training_file = "../data/complex_words_training.txt"
    development_file = "../data/complex_words_development.txt"
    test_file = "../data/complex_words_test_unlabeled.txt"
    train_data = load_file(training_file)
    

FileNotFoundError: ignored

In [0]:
a = np.array([np.array([a,b])for a,b in enumerate([1,2,3,4])])
print(a)

In [0]:
train_file_name = data_path+'/complex_words_training.txt'
dev_file_name = data_path+'/complex_words_development.txt'
test_file_name = data_path+'/complex_words_test_unlabeled.txt'
all_complex(train_file_name)
all_complex(dev_file_name)

In [0]:
word_length_threshold(train_file_name,dev_file_name)

In [0]:
counts = load_ngram_counts(data_path+ngram_path)
avg = sum([val for val in counts.values()])/len(counts)
avg

In [0]:
counts['the']

In [0]:
ngram_path = '/ngram_counts.txt.gz'

word_frequency_threshold(train_file_name,dev_file_name,counts)


Q 1.3 Classifiers

1.3.1 Naive Bayes classification


In [0]:
naive_bayes(train_file_name,dev_file_name,counts)

Q1.4. Ambiguity


In [0]:
from collections import defaultdict

words,labels = load_file(train_file_name)
words_dict = defaultdict(list)
bad_words = []
for word,label in zip(words,labels):
  if word in words_dict and label not in words_dict[word]:
    bad_words.append(word) 
  words_dict[word].append(label)

print(bad_words)


Q2. Document Classification


Q2.1. Reuters Dataset


2.1.1 Explore how many documents are in the dataset, how many categories, how many documents per categories, provide mean and standard deviation, min and max. (use the pandas library to explore the dataset, use the dataframe.describe() method.)


**Answer :**
-
number of topics:  445
number of documents:  19716
                  
count    445.000000
mean      89.871910
std      644.656909
min        1.000000
25%        3.000000
50%        8.000000
75%       37.000000
max    12542.000000

 
count    445.000000
mean      89.871910
std      644.656909
min        1.000000
25%        3.000000
50%        8.000000
75%       37.000000
max    12542.000000



2.1.2 Explore how many characters and words are present in the documents of the dataset.


*By exploring the dataset we looked at the body of the articles.


total words: 2498237
total chars: 15089138

2.1.3 Explain informally what are the classifiers that support the "partial-fit" method discussed in the code.

**SGDClassifier**: A linear classifier such as SVM or logistic regression that is trained with SGD algorithm, thus it can support training on mini-batches.

**Perceptron**: A linear classifier which updates its weights iteratively for each example (batch of size 1)

**NB Multinomial**: The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). 

**Passive-Agressive**:  The name comes from the fact that we want to reduce loss quickly (aggressive) but we also do not want to overcorrect our weights (passive).

2.1.4 Explain what is the hashing vectorizer used in this tutorial. Why is it important to use this vectorizer to achieve "streaming classification"?


In [1]:
import nltk
nltk.download('conll2002')

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


True

In [0]:
from nltk.corpus import conll2002


etr = list(conll2002.iob_sents('esp.train')) # In Spanish
eta = list(conll2002.iob_sents('esp.testa')) # In Spanish
etb = list(conll2002.iob_sents('esp.testb'))# In Spanish

dtr = list(conll2002.iob_sents('ned.train')) # In Dutch
dta = list(conll2002.iob_sents('ned.testa')) # In Dutch
dtb = list(conll2002.iob_sents('ned.testb')) # In Dutch

In [24]:
etr[3]

[('La', 'DA', 'O'),
 ('petición', 'NC', 'O'),
 ('del', 'SP', 'O'),
 ('Abogado', 'NC', 'B-PER'),
 ('General', 'AQ', 'I-PER'),
 ('tiene', 'VMI', 'O'),
 ('lugar', 'NC', 'O'),
 ('después', 'RG', 'O'),
 ('de', 'SP', 'O'),
 ('que', 'CS', 'O'),
 ('un', 'DI', 'O'),
 ('juez', 'NC', 'O'),
 ('del', 'SP', 'O'),
 ('Tribunal', 'NC', 'B-ORG'),
 ('Supremo', 'AQ', 'I-ORG'),
 ('del', 'SP', 'O'),
 ('estado', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('Victoria', 'NC', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 ('se', 'P0', 'O'),
 ('viera', 'VMS', 'O'),
 ('forzado', 'AQ', 'O'),
 ('a', 'SP', 'O'),
 ('disolver', 'VMN', 'O'),
 ('un', 'DI', 'O'),
 ('jurado', 'NC', 'O'),
 ('popular', 'AQ', 'O'),
 ('y', 'CC', 'O'),
 ('suspender', 'VMN', 'O'),
 ('el', 'DA', 'O'),
 ('proceso', 'NC', 'O'),
 ('ante', 'SP', 'O'),
 ('el', 'DA', 'O'),
 ('argumento', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('la', 'DA', 'O'),
 ('defensa', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('que', 'CS', 'O'),
 ('las', 'DA', 'O'),
 (

3.1.2 Train the model using a logistic regression classifier and experiment with better features - looking at the tags of the previous word, the previous word and the following word (add padding words in the vectorizer)

> Indented block



In [0]:
from urllib.parse import urlparse
import string
puncts = list(string.punctuation) + ['...']

def get_ort(w):
  if w.isdigit():
    return 'number'
  elif any([char.isdigit() for char in w]):
    return 'contains-digit'
  elif w in puncts:
    return 'punctuation'
  elif ('-' in w):
    return 'contains-hyphen'
  elif w.istitle():
    return 'capitalized'
  elif w.isupper():
    return 'all-capitals'
  elif (urlparse(w).scheme) and (urlparse(w).netloc):
    return 'URL'
  else:
    return 'regular'

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]


    features = {
        'word-form': word.lower(),
        'ORT': get_ort(word),
        'POS': postag,
        'suffix3' : word[-3:],
        'suffix2' : word[-2:],
        'suffix1': word[-1:],
        'prefix3': word[:3],
        'prefix2': word[:2],
        'prefix1': word[:1],
    }

    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            'prev-word-form' : word1.lower(),
            'prev-POS' : postag1,
        })
    else:
        features.update({
            'prev-word-form' : 'BOS',
            'prev-POS' : '<PAD>',
        })
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            'next-word-form' : word1.lower(),
        })
    else:
        features.update({
            'next-word-form': 'EOS'
            })
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

In [8]:
expected = ['number',
 'contains-digit',
 'punctuation',
 'contains-hyphen',
 'all-capitals',
 'capitalized',
 'URL',
 'regular']

expected == [get_ort(w) for w in ['123','w1',',','hi-hi','SDA','Hello','http://www.google.com','just_word']]

True

In [9]:
sen_triplets = etr[100]
print(' '.join([triplet[0] for triplet in sen_triplets]))
print('-----------------')

word2features(sen_triplets,0)

Imagínense ustedes que entre aquellos españoles , que fueron quienes llevaron a Europa esos dones americanos , se hubiera impuesto la patriotería gastronómica : patatas y tomates se hubieran quedado en curiosidades botánicas .
-----------------


{'ORT': 'capitalized',
 'POS': 'VMM',
 'next-word-form': 'ustedes',
 'prefix1': 'I',
 'prefix2': 'Im',
 'prefix3': 'Ima',
 'prev-POS': '<PAD>',
 'prev-word-form': 'BOS',
 'suffix1': 'e',
 'suffix2': 'se',
 'suffix3': 'nse',
 'word-form': 'imagínense'}

In [17]:
from enum import Enum  
sets = Enum('sets', ['tr_esp','v_esp','t_esp','tr_dut','v_dut','t_dut'], start=0)

sets.t_dut.value

5

In [0]:



from functools import reduce
flatten = lambda l: reduce(lambda x,y: x+y,l)

from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)



data_set = [etr,eta,etb,dtr,dta,dtb]

x_data_set = list(map(lambda x: flatten([sent2features(s) for s in x]),data_set))

y_data_set = list(map(lambda x: flatten([sent2labels(s) for s in x]),data_set))


X_train_esp = x_data_set[sets.tr_esp.value]
y_train_esp = y_data_set[sets.tr_esp.value]

X_train_esp = x_data_set[sets.v_esp.value]
y_valid_esp = y_data_set[sets.v_esp.value]

X_test_esp = x_data_set[sets.t_esp.value]
Y_test_esp = y_data_set[sets.t_esp.value]

X_train_dutch = x_data_set[sets.tr_dut.value]
y_train_dutch = y_data_set[sets.tr_dut.value]

X_valid_dutch = x_data_set[sets.v_dut.value]
y_valid_dutch = y_data_set[sets.v_dut.value]

X_test_dutch = x_data_set[sets.t_dut.value]
Y_test_dutch = y_data_set[sets.t_dut.value]





In [25]:
X_train_esp[0]
y_train_esp[0]

'B-LOC'

In [0]:
from sklearn.linear_model import LogisticRegression

X = v.fit_transform(X_train_esp)

print(X.shape[0])
print(len(Y))
Y = y_train_esp


clf = LogisticRegression(random_state=0).fit(X, Y)
clf.predict(X[0])
